In [166]:
# encoding: utf-8
import pandas as pd
from pandas import *
import datetime
import json
from pymongo import MongoClient
from collections import defaultdict

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

client = MongoClient('localhost', 27017)
db = client.futures
indexMarket = db.indexMarket
peaks = db.peaks
unit=db.unit

db.peaks.delete_many ({})
start='20190101'
indexMarket = DataFrame(list(indexMarket.find({'date': {'$gte': start}})))
unit = DataFrame(list(unit.find()))
var=unit['variety']
for i in set(var):
    try:
        df = indexMarket[indexMarket['variety'] == i]
#         maxs=df.set_high.idxmin()#最小值的索引
#         print(maxs)
        #         date=df[['date'][-1]]
        df = df[['date', 'variety', 'set_open', 'set_close', 'set_high', 'set_low']]
        df.set_index('date', inplace=True)
        maxs = df[['set_high']].stack().max()
#         print(maxs)
        
        mins = df[[ 'set_low']].stack().min()
        gains = round(((maxs / mins-1) * 100), 2)
#         print(gains)
        lesses = round(((mins / maxs-1) * 100), 2)
#         print(lesses)
        peak = (lambda x: gains if mins < df['set_close'][-1] else lesses)(1)
        print(peak)
        df2 = df.copy()
        df2['maxs'] = maxs
        df2['mins'] = mins
        df2['peak'] = peak
        df2 = df2.reset_index()
        begin =df2.date[1]
        df2 = df2[['date','variety', 'maxs', 'mins', 'set_close', 'peak']][-1:]
        peaks.insert_many(json.loads(df2.T.to_json()).values())
#         print('写入数据成功')
#         print(json.loads(df2.T.to_json()).values())
    except:
        print('error')       
df=DataFrame(list(peaks.find()))
df=df[['date','variety','maxs','mins','set_close','peak']]
df=df.set_index('date')
df.sort_values(by="peak",ascending= False)



11.24
28.31
11.24
44.58
16.99
28.12
32.82
18.29
22.55
10.38
7.73
5.74
9.27
9.9
9.82
10.79
31.27
16.47
21.12
7.79
9.91
11.71
17.03
20.78
9.79
30.36
54.04
13.32
-2.0
19.5
15.56
23.38
12.92
41.38
17.69
16.44
18.9
13.5
15.87
25.43
8.23
11.96
10.56
1.01
16.69
9.22
24.0
16.25
24.1
18.06
27.13
22.81
0.0
13.39
26.71
17.9
6.92
11.99


,variety,maxs,mins,set_close,peak
date,,,,,
20191025,I,878.0,570.0,631.0,54.04
20191025,FU,3000.0,2075.0,2105.0,44.58
20191025,NI,149080.0,105445.0,132534.0,41.38
20191025,AP,9769.0,7355.0,7909.0,32.82
20191025,AG,4845.0,3691.0,4425.0,31.27
20191025,FB,73.0,56.0,73.0,30.36
20191025,J,2221.0,1731.0,1792.0,28.31
20190722,RI,2980.0,2326.0,2387.0,28.12
20191025,WR,4574.0,3598.0,3747.0,27.13


In [ ]:
#幅度排序
df=DataFrame(list(peaks.find().sort('peak',-1)))
df=df[['date','variety','maxs','mins','set_close','peak']]
df=df.set_index('date')
    
df['begin']=begin
# df=pd.date_range(start=start,periods=20)
df

In [77]:

#删除表（集合）
peaks.drop()